# CORD-19 Challenge Solution (Team 1-1): Deep Learning Based Question Answering for COVID-19
## Authors: Diego Saldana, Giovanni d'Ario, Solveig Badillo, Sowmini Arun

# Introduction

Our solution aims to be generalizable to new questions, effective within the constraints of a simple python notebook, but also capable of scaling up to a large scale environment. We favored generalizability over specificity because we felt that developing one specific fit-for-purpose solution for each question may give accurate results, but it cannot scale to the real world scenario, where researchers will have many new questions as they learn more about the virus and the disease. We also chose to use open source packages implementing advanced deep learning NLP algorithms that can be used flexibly within a simple notebook but can scale to large scale operations, since they will enable this solution to be used more effectively in institutions that have the computing power to extract its full potential.

We define the problem as follows. We have a set of documents which we consider a 'context' space from which we would like to extract answers. This can be split in three steps, for any given question:

- Retrieving relevant documents / sections from the full text papers
- Locating potential answers from these relevant documents
- Presenting the results to a researcher with enough information

We felt that by having a solution that addresses these three needs, we could dramatically speed up the process that researchers have to follow when looking for answers to their questions. More importantly, we felt this could be generalized to many other questions researchers may have about COVID-19, and SARS-CoV-2.

# Installing the Environment

First, we need to install some packages in order to enable some needed functionality for this notebook, in particular around question answering.

In [ ]:
!pip install "git+https://github.com/deepset-ai/FARM.git@0749dcb8fb46dace0d0987d6fdedf5f28120a461"
!pip install "git+https://github.com/deepset-ai/haystack.git@0cffc6cb1df8782bbf1bbc336807de355687584b"
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

# Function and Class Definitions

Next, we define here a set of functions and classes that we will be using later on in the notebook. These functions allow us to iterate through the data as well as to put in place a question answering system. Finally, we also created some functions to generate HTML to visualize the results of the question answering system.

In [ ]:
import logging
from haystack import Finder
from haystack.database.sql import SQLDocumentStore
from haystack.retriever.tfidf import TfidfRetriever
from haystack.reader.farm import FARMReader
from haystack.utils import print_answers
import json
import os
from pathlib import Path
import re
import tqdm
from xml.etree import ElementTree as ET
from zipfile import ZipFile

logger = logging.getLogger("haystack")
logger.setLevel(logging.ERROR)

def get_full_texts(data_directory):
    """
    Returns a generator that yields dictionaries for each full text.
   
    :param data_directory: The directory where the data is.
    :type data_directory: str
    ...
    :returns: A generator of dict objects.
    :rtype: generator
    """
    if not os.path.isdir(data_directory):
       raise IOError("""The data directory was not found. Download the data from:
            "https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge/download
            and place it under a directory called '%s'.""" % data_directory)
    paths = [os.path.join(l[0], p) for l in os.walk(data_directory) for p in
        l[-1] if p[-5:] == ".json"]
    for path in tqdm.tqdm(paths):
        data = json.load(open(path, "r"))
        yield data
        
def get_authors(full_text):
    """
    Returns a list of formatted authors extracted from the full text dict object.
    
    :param full_text: A full text dict as returned by 'get_full_texts'.
    :type full_text: dict
    ...
    :returns: A list of author strings.
    :rtype: list
    """
    authors = []
    for author in full_text["metadata"]["authors"]:
        author = "".join([c for c in author["first"][:1]] + author["middle"]) + " " + author["last"]
        authors.append(author)
    return authors

def get_sections(full_text, abstracts_only=False):
    """
    Returns a generator that yields (section name, text) tuples for each
    section in a full text dictionary.
   
    :param full_text: A full text dict as returned by 'get_full_texts'.
    :type full_text: dict
    :param abstracts_only: If True, only abstracts are indexed. Defaults to False.
    :type abstracts_only: bool
    ...
    :returns: A generator of (section, text) tuples.
    :rtype: generator
    """
    
    sections = []
    if "abstract" in full_text.keys():
        sections += full_text["abstract"]
    if not abstracts_only:
        if "body_text" in full_text.keys():
            sections += full_text["body_text"]
    for section in sections:
        yield section["section"], section["text"]

def write_documents_to_db(document_store, document_dir, clean_func=None, only_empty_db=False, 
    split_paragraphs=False, abstracts_only=False):
    """
    Writes documents to a sqlite database.
    
    :param document_store: A SQLDocumentStore to store section data in.
    :type document_store: SQLDocumentStore.
    :param document_dir: The directory where the documents are.
    :type document_dir: str
    :param abstracts_only: Only index abstracts.
    :type abstracts_only: bool
    """
    # check if db has already docs
    if only_empty_db:
        n_docs = document_store.get_document_count()
        if n_docs > 0:
            print(f"Skip writing documents since DB already contains {n_docs} docs ...  "
                        "(Disable `only_empty_db`, if you want to add docs anyway.)")
            return None

    # read and add docs
    docs_to_index = []
    count = 1
    for full_text in get_full_texts(document_dir):
        title = full_text["metadata"]["title"].strip()
        if title == "":
            title = "[No title available]"
        authors = ", ".join(get_authors(full_text))
        for section, text in get_sections(full_text, abstracts_only=abstracts_only):
            
            # Skip if not talking about SARS-CoV-2 or COVID-19
            if len(re.findall("(SARS.CoV.2|COVID.19|2019.nCoV)", text, flags=re.IGNORECASE)) == 0:
                continue
            
            docs_to_index.append({
                "text": text, 
                "id": count, 
                "name" : "%s|||%s|||%s|||%s" % (title, section, authors, full_text["paper_id"]), 
            })
            count += 1
            
            if len(docs_to_index) == 500:
                document_store.write_documents(docs_to_index)                
                docs_to_index = []
                
    if len(docs_to_index) > 0:
        document_store.write_documents(docs_to_index)
    print(f"Wrote {count} docs to DB")

def get_finder(data_directory, abstracts_only=False):
    """
    Returns a haystack Finder object with a TfIdfRetriever and a FARMReader based on 
    Transformers.
    
    :param data_directory: The directory where the data is located.
    :type data_directory: str
    :param abstracts_only: If true, only the abstracts section is indexed.
    :type abstracts_only: bool
    ...
    :return: A haystack Finder.
    :rtype: Finder
    """
    document_store = SQLDocumentStore(url="sqlite:////kaggle/working/qa.db")
    write_documents_to_db(document_store=document_store, 
        document_dir=data_directory, only_empty_db=True, 
        abstracts_only=abstracts_only)
    retriever = TfidfRetriever(document_store=document_store)
    reader = FARMReader(
        model_name_or_path="deepset/bert-large-uncased-whole-word-masking-squad2", 
        use_gpu=True)
        #model_name_or_path="gdario/biobert_bioasq", 
        #use_gpu=False)
    finder = Finder(reader, retriever)
    return finder

class Result(object):
    """
    A data structure to store question answering results (at document-section level).
    """
    def __init__(self, title, url, authors, section_title, text, spans):
        """
        See Result class.
        
        :param title: The title of the document.
        :type title: str
        :param url: URL of the document.
        :type url: str
        :param authors: A string with all the authors in the document.
        :type authors: str
        :param section_title: The title of the section.
        :type section_title: str
        :param text: The text of the section.
        :type text: str
        :param spans: A list of span dicts.
        """
        self.title = title
        self.url = url
        self.authors = authors
        self.section_title = section_title
        self.text = text
        self.spans = spans

def get_results(finder, top_k_retriever, top_k_reader, candidate_doc_ids, question):
    """
    Builds a list of Result objects for a given question.
    
    :param finder: A haystack Finder instance.
    :type finder: Finder
    :param top_k_retriever: The number of top document-sections obtained by the retriever.
    :type top_k_retriever: int
    :param top_k_reader: The number of top answers obtained by the reader.
    :type top_k_reader: int
    :param candidate_doc_ids: A list of doc ids to filter on.
    :type candidate_doc_ids: list
    :param question: A question to be answered.
    :type question: str
    ...
    :return: A list of Result instances.
    :rtype: list
    """
    paragraphs, meta_data = finder.retriever.retrieve(question, top_k=top_k_retriever, candidate_doc_ids=candidate_doc_ids)
    results = []

    if len(paragraphs) > 0:

        # 3) Apply reader to get granular answer(s)
        len_chars = sum([len (p) for p in paragraphs])
        predictions = finder.reader.predict(question=question,
            paragraphs=paragraphs,
            meta_data_paragraphs=meta_data,
            top_k=top_k_reader)

        # Add corresponding document_name if an answer contains the document_id (only supported in FARMReader)
        for prediction in predictions["answers"]:
            document = finder.retriever.document_store.get_document_by_id(prediction["document_id"])
            title, section_title, authors, paper_id = document["name"].split("|||")
            url = "https://cord-19.apps.allenai.org/paper/%s" % paper_id
            spans = [{
                "start" : prediction["offset_start"], 
                "end" : prediction["offset_end"], 
            }]
            result = Result(
                title, 
                url, 
                authors, 
                section_title, 
                prediction["context"], 
                spans, 
            )
            results.append(result)
    
    return results

def add_search_result_element(container, result):
    """
    Adds a search result node to an html container.
    
    :param container: An ElementTree node.
    :type container: ElementTree
    :param result: A result to be added.
    :type result: Result
    """
    # Title
    div = ET.SubElement(container, "div")
    a = ET.SubElement(div, "a", href=result.url, target="_blank")
    a.text = result.title

    # Authors
    div = ET.SubElement(container, "div")
    b = ET.SubElement(div, "b")
    b.text = result.authors

    # Section Title
    div = ET.SubElement(container, "div")
    b = ET.SubElement(div, "b", style="color: grey;")
    b.text = result.section_title
    
    # Snippet
    cursor = 0
    for span in result.spans:
        div = ET.SubElement(container, "div")
        p = ET.SubElement(div, "p")
        span_element = ET.SubElement(p, "span")
        span_element.text = result.text[:span["start"]]
        span_element = ET.SubElement(p, "span", style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;")
        span_element.text = result.text[span["start"]:span["end"]]
        cursor = span["end"]
    if cursor < len(result.text):
        span_element = ET.SubElement(p, "span")
        span_element.text = result.text[cursor:]

def generate_html(question, results):
    """
    Generate HTML to display for a given question and a list of results.
    
    :param question: The question that was asked.
    :type question: str
    :param results: A list of Result instances.
    :type results: list
    ...
    :return: HTML content showing the question and the results (with links).
    :rtype: str
    """
    container = ET.Element("div")
    
    # Add question
    div = ET.SubElement(container, "div")
    h = ET.SubElement(div, "h1")
    h.text = question
    
    # Add answers
    for result in results:
        add_search_result_element(container, result)
        ET.SubElement(container, "hr")
    html = str(ET.tostring(container))[2:-1]
    return html

In [ ]:
## Delete the database if needed
#
#!rm /kaggle/working/qa.db

# Setting up the Question Answering System

We are using [Haystack](https://github.com/deepset-ai/haystack) to index the CORD-19 dataset, retrieve relevant documents as well as answers from a given question. In order to retrieve the documents, we chose to use the TfIdf Retriever because it is easier to implement in a simple notebook. However, in a large scale real-world application, we would recommend using an ElasticSearch retriever and server. We also chose to use a BERT Transformer model as the Reader, which will retrieve potential answers.

This process will take some time, as Haystack will index every section of every document in the CORD-19 dataset that mentions either SARS-CoV-2, COVID-19 and/or 2019-nCoV. This choice was made in order to make sure the solution works in the Kaggle environment. It will also download a BERT model checkpoint and load it to be used as the Reader in Haystack.

In [ ]:
finder = get_finder("/kaggle/input/", abstracts_only=False)

# Listing Our Questions

We created a list of questions that Haystack can use to retrieve documents and then locate answers on. This makes our solution very flexible, since all it takes is adding more questions in the list and re-running the notebook to get answers to new questions.

As mentioned before, we're using a TfIdf Retriever, which is less effective than an ElasticSearch based retriever. For this reason, we decided to retrieve 2000 documents rather than a small number. This means the BERT model will have to run over more documents, and so it will be slower. However, in a real world scenario, using ElasticSearch may result in better answers with less documents.

We also chose to keep the top 5 potential answers. However, we can easily change that by changing the value of the top_k_reader parameter in the cell below.

In [ ]:
top_k_retriever = 2000
top_k_reader = 5
candidate_doc_ids = None
questions = [
    
    # Range of incubation periods for the disease in humans (and how this varies across age and health status) and how long individuals are contagious, even after recovery.
    "What is the incubation period of SARS-CoV-2?", 
    
    # Prevalence of asymptomatic shedding and transmission (e.g., particularly children).
    "What is the prevalence of asymptomatic transmission of SARS-CoV-2?", 
    
    # Seasonality of transmission.
    "Is the transmission of SARS-CoV-2 seasonal?", 
    "Is the transmission of SARS-CoV-2 influenced by humidity?", 
    
    # Physical science of the coronavirus (e.g., charge distribution, adhesion to hydrophilic/phobic surfaces, environmental survival to inform decontamination efforts for affected areas and provide information about viral shedding).
    "How are the electrostatics on the surface of coronavirus?", 
    "How does coronavirus adhere to hydrophobic or hydrophilic surfaces?", 
    "What factors influence survival of coronavirus in the environment?", 
    
    # Persistence and stability on a multitude of substrates and sources (e.g., nasal discharge, sputum, urine, fecal matter, blood).
    "Does the virus persist on nasal discharge?", 
    "Does the virus persist on sputum?", 
    "Does the virus persist on urine?", 
    "Does the virus persist on fecal matter?", 
    "Does the virus persist on blood?",
    
    # Persistence of virus on surfaces of different materials (e,g., copper, stainless steel, plastic).
    "Which materials does the virus persist on?", 
    
    # Natural history of the virus and shedding of it from an infected person
    "How do people get infected with SARS-CoV-2?", 
    "How do infected people transmit SARS-CoV-2?", 

    # Implementation of diagnostics and products to improve clinical processes
    "What diagnostics have been implemented for SARS-CoV-2?", 
    
    # Disease models, including animal models for infection, disease and transmission
    "What models for infection exist for SARS-CoV-2?", 
    "What models for transmission exist for SARS-CoV-2?",  
    
    # Tools and studies to monitor phenotypic change and potential adaptation of the virus
    "What studies exist about monitoring phenotypic change in SARS-CoV-2?", 
    "What studies exist about monitoring adaptation of SARS-CoV-2?", 
    
    # Immune response and immunity
    "How is the immune response to SARS-CoV-2?", 
    "How is immunity to SARS-CoV-2 developed?", 
    
    # Effectiveness of movement control strategies to prevent secondary transmission in health care and community settings
    "Are movement control strategies effective to prevent secondary transmission of SARS-CoV-2?", 
    
    # Effectiveness of personal protective equipment (PPE) and its usefulness to reduce risk of transmission in health care and community settings
    "Is personal protective equipment effective at reducing risk of transmission of SARS-CoV-2?", 
    
    # Role of the environment in transmission
    "Is transmission of SARS-CoV-2 dependent on environmental factors?", 
]

# Computing and Displaying the Results

Finally, running the cell below will compute the answers and display them. We created a function (generate_html) that generates results similar to those that would be found in a search engine. This displays for each answer:

- The title of the paper (which is also a link to it)
- The authors
- A piece of text where the potential answer was found (with highlights)

This will take some time, due to the fact that we retrieve 2000 documents, however this would be much faster in a real world setting by using an Elastic Search retriever.

In [ ]:
from IPython.core.display import display
from IPython.core.display import HTML
from IPython.utils import io

html_string = ""
for question in questions:
    with io.capture_output() as captured:
        results = get_results(finder, top_k_retriever, top_k_reader, candidate_doc_ids, question)
    current_html_string = generate_html(question, results)
    html_string += current_html_string
    display(HTML(current_html_string))

In [ ]:
## Display html, then copy and paste as pre-computed outputs if desired
#print(html_string)

# Pre-computed Outputs

We show the pre-computed outputs below. The outputs are also shown on the cell above, although they seem to be hidden by default (perhaps because they are very long). We recommend running this notebook using a GPU, as otherwise it may take a very long time to compute all the outputs.

<div><div><h1>What is the incubation period of SARS-CoV-2?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/81191a43e52981527457fa8003a0e2cc6b7b6442" target="_blank">First known person-to-person transmission of severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) in the USA</a></div><div><b>I Ghinai, TD Mcpherson, JC Hunter, HL Kirking, D Christiansen, K Joshi, R Rubin, S Morales-Estrada, SR Black, M Pacilli, MJ Fricchione, RK Chugh, KA Walblay, S Ahmed, WC Stoecker, NF Hasan, DP Burdsall, HE Reese, M Wallace, C Wang, D Moeller, J Korpics, SA Novosad, I Benowitz, MW Jacobs, VS Dasari, MT Patel, J Kauerauf, M Charles, NO Ezike, V Chu, CM Midgley, MA Rolfes, SI Gerber, X Lu, S Lindstrom, JR Verani, JE </b></div><div><b style="color: grey;">Discussion</b></div><div><p><span>rtPCR might not be sufficient to definitively rule out infection over a </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">14-day </span><span>incubation period, and only a convenience sample of a minority of healt</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/14083b334f48819d6a42a2ed917adfa200c59df4" target="_blank">Current State and Predicting Future Scenario of Highly Infected Nations for COVID-19 Pandemic</a></div><div><b>NL Patil</b></div><div><b style="color: grey;">Introduction</b></div><div><p><span>e contacts (within about 6 feet) [1] and has mean incubation period of </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">6.4 days </span><span>with range of 2.1 to 11.1 days [2] . Asymptotic spread has made detect</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/6314943df23e3c79ded21bd2d5ced6bfd35be893" target="_blank">[No title available]</a></div><div><b /></div><div><b style="color: grey;">Clinical disease course of COVID-19</b></div><div><p><span>The official incubation period for SARS-CoV-2 is </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">2-14 days </span><span>and therefore 14 days is the chosen cut-off for self-quarantine. Guan = 138) showed that th</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/PMC7138423" target="_blank">A Comprehensive Literature Review on the Clinical Presentation, and Management of the Pandemic Coronavirus Disease 2019 (COVID-19)</a></div><div><b>A Muacevic, JR Adler, P Kakodkar, N Kaka, M Baig</b></div><div><b style="color: grey;">Review</b></div><div><p><span>The official incubation period for SARS-CoV-2 is </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">2-14 days </span><span>and therefore 14 days is the chosen cut-off for self-quarantine. Guan et al. (n = 1324) est</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/PMC7096724" target="_blank">Prevalence of Underlying Diseases in Hospitalized Patients with COVID-19: a Systematic Review and Meta-Analysis</a></div><div><b>A Emami, F Javanmardi, N Pirbonyeh, A Akbari</b></div><div><b style="color: grey;">Discussion</b></div><div><p><span>as follows: diagnosis, mode of transmission, long incubation period (</span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">3 to 14 days</span><span>), predicting the number of infected cases in the community, and insu</span></p></div><hr /></div><div><div><h1>What is the prevalence of asymptomatic transmission of SARS-CoV-2?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/4c84d1153daa335f5ff6d61db665024b32ac8195" target="_blank">Clinical Characteristics of SARS-CoV-2 Pneumonia Compared to Controls in Chinese Han Population</a></div><div><b>Y Xu, Y Li, Q Zeng, Z Lu, Y Li, W Wu, S Dong, G# Huang, X Wang, DY Xu, YR Li, Q Zeng, ZB Lu, YZ Li</b></div><div><b style="color: grey;">Introduction</b></div><div><p><span>11, 2020, the Chinese Center for Disease Control and Prevention (China CDC) has officially reported that there are </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">2.0% </span><span>(889) asymptomatic cases, 2.3%</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/4c96d9cfc6da83e41b0889579d1e8b25765bd355" target="_blank">Mortality of COVID-19 is Associated with Cellular Immune Function Compared to Immune Function in Chinese Han Population</a></div><div><b>Q Zeng, Y Li, G Huang, W Wu, S Dong, Y Xu</b></div><div><b style="color: grey;">Introduction</b></div><div><p><span>11, 2020, the Chinese Center for Disease Control and Prevention (China CDC) has officially reported that there are </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">2.0% </span><span>(889) asymptomatic cases, 2.3%</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/320e1640a5ddfcbed21e1afff08a86dc1b0af5b2" target="_blank">Cardiovascular Considerations for Patients, Health Care Workers, and Health Systems During the Coronavirus Disease 2019 (COVID-19) Pandemic</a></div><div><b>E Driggin, MV Madhavan, B Bikdeli, T Chuich, J Laracy, G Bondi-Zoccai, TS Brown, C Der Nigoghossian, DA Zidar, J Haythe, D Brodie, JA Beckman, AJ Kirtane, GW Stone, HM Krumholz, SM , SA Parikh</b></div><div><b style="color: grey;">Pathophysiology, Epidemiology, and Clinical Features of COVID-19</b></div><div><p><span>ated by asymptomatic infection in a sizable portion of individuals (</span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">as many as 20%</span><span>), which may significantly contribute to further spread of infection</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/e32a6e93ada54e9c2fc770540b077009a20228c6" target="_blank">Comment</a></div><div><b /></div><div><b style="color: grey;">Scientific and ethical basis for social-distancing interventions against COVID-19</b></div><div><p><span>sic reproduction number [R 0 ] of 1&#8226;5, 2&#8226;0, or 2&#8226;5) and assumed </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">between 7&#8226;5% and 50&#8226;0% </span><span>of infections were asymptomatic. The interventions were quarant</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/2278d4bb9e0ab0b4d8a77924cf165870bd660b30" target="_blank">Epidemiological and Clinical Characteristics of Children with Coronavirus Disease 2019</a></div><div><b>Q Wu, Y Xing, L Shi, W Li, Y Gao, S Pan, Y Wang, W Wang, Q Xing, W Wang&#65306;qingdao</b></div><div><b style="color: grey;">Discussion</b></div><div><p><span>Except for one critically ill patient, 20 (</span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">27.03%</span><span>) cases were asymptomatic carriers of SARS-CoV-2, 53 (71.62%) were mild to moderate cases with variou</span></p></div><hr /></div><div><div><h1>Is the transmission of SARS-CoV-2 seasonal?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/ce0c0fb84dcfa0b283c542d6024c44eb8ddd16d5" target="_blank">Susceptible supply limits the role of climate in the COVID-19 pandemic</a></div><div><b>RE Baker, W Yang, GA Vecchi, C Jessica, E Metcalf, BT Grenfell</b></div><div><b style="color: grey;">Discussion</b></div><div><p><span> the disease will likely be tied to climate factors and </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">seasonal peaks may vary with latitude </span><span>(Fig.S3) . A more detailed understanding of climate driv</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/e1200121af466e437f3cf17aa10654b7725edb07" target="_blank">[No title available]</a></div><div><b>C Massonnaud, J Roux, P Cr&#233;pey</b></div><div><b style="color: grey;">Discussion</b></div><div><p><span> the following winter. However, </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">whether SARS-CoV-2 transmission will be affected by seasonal variations remains unclear</span><span>. Although many infectious dise</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/e7a4159ce5cead74ff87dc1199e3f461631adf96" target="_blank">Journal Pre-proof Diarrhea during COVID-19 infection: pathogenesis, epidemiology, prevention and management Diarrhea during COVID-19 infection: pathogenesis, epidemiology, prevention and management</a></div><div><b>FD &amp;apos; Amico, DC Baumgart, S Danese, L Peyrin-Biroulet, F D&amp;apos;amico,  Laurent Peyrin-Biroulet</b></div><div><b style="color: grey;">Discussion</b></div><div><p><span>SARS-CoV-2 is similar to SARS-CoV and MERS-CoV, but </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">it is transmitted faster than its predecessors </span><span>and for this reason a rapid and optimal diagnostic </span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/88bdd99c41f943dd769a81f74a53434f3eb4f371" target="_blank">Preliminary evidence that higher temperatures are associated with lower incidence of COVID-19, for cases reported globally up to 29th February 2020</a></div><div><b>M Bannister-Tyrrell, A Meyer, C Faverjon, A Cameron</b></div><div><b style="color: grey;">Figure 1: Distribution of ADM1 units that reported at least COVID-19 case up to 29th February 2020.</b></div><div><p><span>This study provides preliminary evidence that </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">there may be seasonal variability </span><span>in transmission of SARS-CoV-2, but this analysis does not imply that t</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/df97f804b68dcf16ffcc3c2c0894528d6d0c7ff2" target="_blank">Articles The effect of control strategies to reduce social mixing on outcomes of the COVID-19 epidemic in Wuhan, China: a modelling study</a></div><div><b>K Prem, Y Liu, TW Russell, AJ Kucharski, RM Eggo, N Davies</b></div><div><b style="color: grey;">Discussion</b></div><div><p><span> when physical distancing measures are relaxed. </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">The effects of seasonality on SARS-CoV-2 are difficult </span><span>to predict without long time series; supporting</span></p></div><hr /></div><div><div><h1>Is the transmission of SARS-CoV-2 influenced by humidity?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/f53c2ce5cf9433c30694ec178d0b6a16c0196c51" target="_blank">Symptom-Based Isolation Policies: Evidence from a Mathematical Model of Outbreaks of Influenza and COVID-19</a></div><div><b>A Burns, A Gutfraind</b></div><div><b style="color: grey;">Contact Rates</b></div><div><p><span> higher in cold and dry weather, but unlike influenza, </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">it is also affected by absolute humidity </span><span>[58] . Contact rates were also modified during weekend</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/64905c9eeababdc0e26ccf4e970e119c929d5f20" target="_blank">COVID-19 in Colombia endpoints. Are we different, like Europe?</a></div><div><b>P Amariles, J Granados, M Ceballos, CJulio Montoya</b></div><div><b style="color: grey;">Introduction</b></div><div><p><span> due to geographic and climatic conditions; c) </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">less spread of the virus by location in the tropical zone</span><span>; and d) earlier preventive measures to contai</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/e1200121af466e437f3cf17aa10654b7725edb07" target="_blank">[No title available]</a></div><div><b>C Massonnaud, J Roux, P Cr&#233;pey</b></div><div><b style="color: grey;">Discussion</b></div><div><p><span> the following winter. However, </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">whether SARS-CoV-2 transmission will be affected by seasonal variations remains unclear</span><span>. Although many infectious dise</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/4cd306cff8ce22bd079af8ad5750aae567d78126" target="_blank">Journal Pre-proof How to perform tracheal intubation for COVID-19 patients safely and effectively: Experience AND tips from Sichuan, China How to Perform Tracheal Intubation for COVID-19 Patients Safely and Effectively: Experience AND Tips from Sichuan, China</a></div><div><b>J Huang, J Zeng, M Xie, X Huang, X Wei, L Pan, &#65292; Huang</b></div><div><b style="color: grey;" /></div><div><p><span>firmed cases around multiple countries every day. </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">The transmission dynamics is not fully understood </span><span>[1] . Establishing an artificial airway is a techn</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/96f65d71931741ea4668f2fbcae54a8d8905abac" target="_blank">From SARS-CoV to SARS-CoV-2 Outbreak: Similarities in the Early Epidemics and Prediction of Future Trends</a></div><div><b>Z Chen, W Zhang, Y Lu, C Guo, Z Guo, C Liao, X Zhang, Y Zhang, X Han, Q Li, J Lu, H Lu</b></div><div><b style="color: grey;" /></div><div><p><span>ation of the pathogen [ Figure 1E ]. In contrast to that of SARS-CoV, the discovery of human-to-human transmission of SARS-CoV-2 came </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">relatively late</span><span>.</span></p></div><hr /></div><div><div><h1>How are the electrostatics on the surface of coronavirus?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/0cbb22ef36a375f7abca0f173953ba162929b40e" target="_blank">The COVID-19 pandemic: Important considerations for contact lens practitioners</a></div><div><b>L Jones, K Walsh, M Willcox, P Morgan, J Nichols</b></div><div><b style="color: grey;">Contact lens material considerations</b></div><div><p><span>e viruses the major driving factors were electrostatic interactions (</span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">charge driven</span><span>), followed by hydrophobic interactions, with only minor impact from</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/e6d2c030bb4bb71cae43fd29624e70872b1662a4" target="_blank">Epidemiological, clinical and virological characteristics of 74 cases of coronavirus-infected disease 2019 (COVID-19) with gastrointestinal symptoms</a></div><div><b>X Jin, J Lian, J Hu, J Gao, L Zheng, Y Zhang, S Hao, H Jia, X Zhang, G Yu, K Xu, X Wang, J Gu, S Zhang, C Ye,  Jin, Y Lu, X Yu, X Yu, J Huang, K Xu, Q Ni, B Yu, B Zhu, Y Li, J Liu, H Zhao, X Zhang, L Yu, Y , J Su, J Tao, J Lang, X Wu, W Wu, T Qv, D Xiang, P Yi, D Shi, Y Chen, Y Ren, Y Qiu, L Li,  Jifang, Y Yang</b></div><div><b style="color: grey;">sequence alignment and protein model structure analysis</b></div><div><p><span>nges between ZJ01 and Wuhan-Hu-1 are largely concentrated in the </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">mutation zone of S2 </span><span>(figure 1C, green ellipse), where the detailed mechanisms need fu</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/07e833d0917cace550853f72923856d0fe1a7120" target="_blank">Mycroft-West et al. (2020) Running title: SARS-CoV-2 surface S1 Receptor Binding Domain binds heparin The 2019 coronavirus (SARS-CoV-2) surface protein (Spike) S1 Receptor Binding Domain undergoes conformational change upon heparin binding</a></div><div><b>C Mycroft-West, S #1, SElli #2, S #3,  Guimond, G Miller, J Turnbull, E Yates, M Guerrini, D Fernig, M Lima, M Skidmore, &#167; </b></div><div><b style="color: grey;">Abstract</b></div><div><p><span>s including S-associated coronavirus strain HSR1. Here we use </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">surface plasmon resonance </span><span>and circular dichroism to measure the interaction between the </span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/fe69099d9491aa640dd66e8d606651c262755361" target="_blank">Structure analysis of the receptor binding of 2019-nCoV</a></div><div><b>Y Chen, Y Guo, Y Pan, ZJoe Zhao</b></div><div><b style="color: grey;">Introduction</b></div><div><p><span>ike glycoprotein that gives coronaviruses a crownlike appearance by </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">forming spikes </span><span>on their surface. The amino acid sequence of spike glycoprotein con</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/4faf34d795e5ff74a886528e46268af783fe712b" target="_blank">Structure analysis of the receptor binding of 2019-nCoV</a></div><div><b>Y Chen, Y Guo, Y Pan, ZJoe Zhao</b></div><div><b style="color: grey;">Introduction</b></div><div><p><span>ike glycoprotein that gives coronaviruses a crownlike appearance by </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">forming spikes </span><span>on their surface. The amino acid sequence of spike glycoprotein con</span></p></div><hr /></div><div><div><h1>How does coronavirus adhere to hydrophobic or hydrophilic surfaces?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/949feecec7c10e71532e860f7f1230bd696e8234" target="_blank">Epitope-based peptide vaccine design and target site characterization against novel coronavirus disease caused by SARS-CoV-2</a></div><div><b>L Li, T Sun, Y He, W Li, Y Fan, J Zhang, B </b></div><div><b style="color: grey;">B-cell epitopes can guide B-cell to recognize and activate defense responses against viral infection.</b></div><div><p><span>with average of 1 , minimum of 0.042, and maximum of 6.051 (Fig. 1D) . </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">Exposed </span><span>on the surface and hydrophilic in nature making beta turn a vital struc</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/522da4df2c31909547ac81763ad28d318fdb88e6" target="_blank">A highly conserved cryptic epitope in the receptor-binding 3 domains of SARS-CoV-2 and SARS-CoV 4 5 6 7</a></div><div><b>M Yuan, NC Wu, X Zhu, CD Lee, RT Y So, H Lv, CK P Mok, IA Wilson</b></div><div><b style="color: grey;">62</b></div><div><p><span>2 and 63 SARS-CoV-2 recognition by CR3022 is largely driven by </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">hydrophobic interactions </span><span>(Fig.   64 1E). Five out of 11 somatic mutations are found in </span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/20c09ed5eeacb1e7cc346d6cee13e46a5284490d" target="_blank">The potential SARS-CoV-2 entry inhibitor</a></div><div><b>W Sec, P Jiapu Rd,  City</b></div><div><b style="color: grey;">Abstract</b></div><div><p><span> SARS-CoV-2 RBD by SwissDock service. Regarding contact modes, </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">hydrophobic interactions </span><span>contribute significantly in binding and additional hydrogen bo</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/20c09ed5eeacb1e7cc346d6cee13e46a5284490d" target="_blank">The potential SARS-CoV-2 entry inhibitor</a></div><div><b>W Sec, P Jiapu Rd,  City</b></div><div><b style="color: grey;">Result</b></div><div><p><span>3B , C and D. Regarding the contact modes by SwissDock server, </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">hydrophobic interactions </span><span>contribute significantly for binding (Fig 3D and E) . The addi</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/48516562e47b75ec5f66456970afb32cb53c805a" target="_blank">[No title available]</a></div><div><b>JC L Rodrigues, SS Hare, A Edey, A Devaraj, J Jacob, A Johnstone, R Mcstay, A Nair, G Robinson</b></div><div><b style="color: grey;">Practical guidance for COVID-19</b></div><div><p><span>COVID-19 viruses, mostly spread by </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">large droplets</span><span>, may remain viable on surfaces for up to 24 h, but will lose infectivity with disinfectants. 18 Each</span></p></div><hr /></div><div><div><h1>What factors influence survival of coronavirus in the environment?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/98222351ab70f5fefc0258b92ac7b66c08f66ff6" target="_blank">The epidemic of 2019-novel-coronavirus (2019-nCoV) pneumonia and insights for emerging infectious diseases in the future</a></div><div><b>J Li, Z You, Q Wang, Z Zhou, Y Qiu, R Luo, X Ge</b></div><div><b style="color: grey;">Clinical symptoms of 2019-nCoV infection</b></div><div><p><span>nment, especially in air-conditioned space, owing to </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">lower ambient temperature and lower humidity </span><span>[19] . However, SARS-CoV had an aberrant trait that </span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/8699664d8100af1e701e25daeb8a4fced18a3daf" target="_blank">The epidemic of 2019-novel-coronavirus (2019-nCoV) pneumonia and insights for emerging infectious diseases in the future</a></div><div><b>J Li, Z You, Q Wang, Z Zhou, Y Qiu, R Luo, X Ge</b></div><div><b style="color: grey;">Clinical symptoms of 2019-nCoV infection</b></div><div><p><span>nment, especially in air-conditioned space, owing to </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">lower ambient temperature and lower humidity </span><span>[19] . However, SARS-CoV had an aberrant trait that </span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/17307ab6eda64e1538da204c02114dd36a9b7fd6" target="_blank">The immune vulnerability landscape of the 2019 Novel Coronavirus, SARS- CoV-2</a></div><div><b>J Zhu, J Kim, X Xiao, Y Wang, D Luo, R Chen, L Xu, H Zhang, G Xiao, JW Schoggins, X Zhan, T Wang, Y Xie</b></div><div><b style="color: grey;">Results and Discussion</b></div><div><p><span /><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">Genetic variations </span><span>can modify the immunogenicity landscape of the virus, and impact its survival fitness. The selection of effective vaccination epito</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/cf0987354fb6a4c2d4ae2353edef93939b4df2c4" target="_blank">[No title available]</a></div><div><b /></div><div><b style="color: grey;">Editorial</b></div><div><p><span>luence by local contextual variables such as </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">hygiene practices, crowding, and infection control standards</span><span>. 8 High-income countries such as the United </span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/16ffae6b21685dfcd71c62398ba65adefc9b5d44" target="_blank">[No title available]</a></div><div><b>JM Clements</b></div><div><b style="color: grey;">Discussion</b></div><div><p><span>eating wild animals will not transmit the disease, </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">living and working in close proximity to animals </span><span>clearly influenced this outbreak and could influen</span></p></div><hr /></div><div><div><h1>Does the virus persist on nasal discharge?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/c7c745598a4d37174b3aaa575abf40c964411aa0" target="_blank">Infection and Rapid Transmission of SARS-CoV-2 in Ferrets</a></div><div><b>Y Kim, S Kim, S Kim, RJ Webby, JU Jung, YKi Choi Correspondence</b></div><div><b style="color: grey;">SUMMARY</b></div><div><p><span>ed in nasal washes and peaked at 4 dpi (3.83 log 10 copies/mL), </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">persisting until 8 dpi </span><span>before dropping below detection limits at 10 dpi (Table 1) . Th</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/9b3c8ae64057fb199a1770ca46929ce031c34b56" target="_blank">Understanding the emerging coronavirus: what it means for health security and infection prevention</a></div><div><b>A Peters, P Vetter, C Guitart, N Lotfinejad, D Pittet</b></div><div><b style="color: grey;">What this means for infection prevention and control</b></div><div><p><span> that hospitals deal with on an annual basis, and </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">transmission seems to occur mainly through droplets</span><span>. Even a high estimate of the R 0 rate is many ti</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/aa12917f77de45b88f1c9a06fba69686863b81ae" target="_blank">Characterization and treatment of SARS-CoV-2 in nasal and bronchial human airway epithelia</a></div><div><b>A Pizzorno1, B Padey1, T Julien1, S Trouillet-Assant1</b></div><div><b style="color: grey;">Main Text:</b></div><div><p><span>mostly unchanged all throughout the infection in bronchial HAE but it is </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">highly upregulated </span><span>in nasal HAE at 48 hpi and onwards ( Fig. 3C and Data S1).</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/a6b0f6b85b36a9f15b8dd7293ff9c68d7f10f932" target="_blank">Clinical Manifestations of Children with COVID-19: a Systematic Review</a></div><div><b>TH De Souza, JA Nadal, RJ N Nogueira, RM Pereira, MB Brand&#227;o, T Henrique De Souza, J Antonio, H Nadal, R Jos&#233;, N Nogueira, R Mendes Pereira, M Barciela</b></div><div><b style="color: grey;">DISCUSSION</b></div><div><p><span>iral shedding in stool and nasal secretions made children </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">facilitators of viral transmission</span><span>. 5, 42 In the study of Xu et al., eight of ten children w</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/ed99ea6c7af5c77db2bf336d263db55646686098" target="_blank">Journal Pre-proof Pay attention to situation of SARS-CoV-2 and TCM advantages in treatment of novel coronavirus infection Pay attention to situation of SARS-CoV-2 and TCM advantages in treatment of novel coronavirus infection</a></div><div><b>C Liu</b></div><div><b style="color: grey;">"Herd Immunity" is not suitable for prevention and control strategies in outbreak of infectious diseases</b></div><div><p><span>om the disease for a long time. </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">Some discharged patients may have positive nucleic acid tests and still carry the virus</span><span>. If so, do you think it is dif</span></p></div><hr /></div><div><div><h1>Does the virus persist on sputum?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/9bbfd3d34ee18ea1b9f4669331a6cee9c5992893" target="_blank">Virological assessment of hospitalized cases of coronavirus disease 2019 *equal contribution **senior authors with equal contribution</a></div><div><b>R W&#246;lfel, VM Corman, W Guggemos, M Seilmaier, S Zange, MA M&#252;ller, D Niemeyer, TC Jones Kelly, P Vollmar, C Rothe, M Hoelscher, T Bleicker, S Br&#252;nink, J Schneider, R Ehmann, K Zwirglmaier, C Drosten, C Wendtner, D; Katrin Zwirglmaier, K M&#252;nchen-Schwabing, G , D; Niemeyer,  Terence, J Kelly</b></div><div><b style="color: grey;">cases, providing proof of active virus replication in upper respiratory tract tissues.</b></div><div><p><span>mples of one same patient. Shedding of viral RNA from sputum </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">outlasted the end of symptoms</span><span>. Seroconversion occurred after 6-12 days, but was not follo</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/9b3c8ae64057fb199a1770ca46929ce031c34b56" target="_blank">Understanding the emerging coronavirus: what it means for health security and infection prevention</a></div><div><b>A Peters, P Vetter, C Guitart, N Lotfinejad, D Pittet</b></div><div><b style="color: grey;">What this means for infection prevention and control</b></div><div><p><span> that hospitals deal with on an annual basis, and </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">transmission seems to occur mainly through droplets</span><span>. Even a high estimate of the R 0 rate is many ti</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/PMC7121749" target="_blank">Evidence of SARS-CoV-2 Infection in Returning Travelers from Wuhan, China</a></div><div><b>S Hoehl, H Rabenau, A Berger, M Kortenbusch, J Cinatl, D Bojkova, P Behrens, B B&#246;ddinghaus, U G&#246;tsch, F Naujoks, P Neumann, J Schork, P Tiarks-Jungk, A Walczok, M Eickmann, MJ.G.T. Vehreschild, G Kann, T Wolf, R Gottschalk, S Ciesek</b></div><div><b style="color: grey;" /></div><div><p><span>transferred to University Hospital Frankfurt for evaluation. However, testing to detect SARS-CoV-2 by RT-PCR of a throat swab and </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">sputum was negative</span><span>.</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/d0e43db0933c7aebb44b455a2b9f60eb8cfa95d4" target="_blank">Geographical Distance to the Epicenter of Covid-19 Predicts the Burnout of Working Population: Ripple effect or Typhoon Eye effect? E-mail Address for Each Author: Declaration of Interest Geographical Distance to the Epicenter of Covid-19 Predicts the Burnout of Working Population: Ripple effect or Typhoon Eye effect?</a></div><div><b>SX Zhang, H Huang, F Wei, SX </b></div><div><b style="color: grey;">Method</b></div><div><p><span>nd Jiangxi; and the rest went further (Kinetz, 2020) . </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">The virus kept on spreading across China</span><span>, and we conducted a survey on February 20-21, 2020 on </span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/3e5edc4ff36064478e209800a15365cd5f710756" target="_blank">First case of Coronavirus Disease 2019 (COVID-19) pneumonia in Taiwan KEYWORDS-NC-ND license (http://creativecommons.org/licenses/by- nc-nd/4.0/)</a></div><div><b>S Cheng, Y Chang, YFan Chiang, Y Chien, M Cheng, C Yang, C Huang, Y Hsu</b></div><div><b style="color: grey;">Case report</b></div><div><p><span>On illness day 17, her sputum turned </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">undetectable </span><span>for SARS-CoV-2 by rRT-PCR assay. The results of subsequent rRT-PCR of oropharyngeal swabs and sputum</span></p></div><hr /></div><div><div><h1>Does the virus persist on urine?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/9bbfd3d34ee18ea1b9f4669331a6cee9c5992893" target="_blank">Virological assessment of hospitalized cases of coronavirus disease 2019 *equal contribution **senior authors with equal contribution</a></div><div><b>R W&#246;lfel, VM Corman, W Guggemos, M Seilmaier, S Zange, MA M&#252;ller, D Niemeyer, TC Jones Kelly, P Vollmar, C Rothe, M Hoelscher, T Bleicker, S Br&#252;nink, J Schneider, R Ehmann, K Zwirglmaier, C Drosten, C Wendtner, D; Katrin Zwirglmaier, K M&#252;nchen-Schwabing, G , D; Niemeyer,  Terence, J Kelly</b></div><div><b style="color: grey;">cases, providing proof of active virus replication in upper respiratory tract tissues.</b></div><div><p><span>ol samples in spite of high virus RNA concentration. Blood and </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">urine never yielded virus</span><span>. Active replication in the throat was confirmed by viral repl</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/cbf20969da0fdadbb513502aa324d940defc5f15" target="_blank">Clinical and virological data of the first cases of COVID-19 in Europe: a case series</a></div><div><b>F , F Lescure, L Bouadma, P Wicky, Q Le Hingrat, J Lucet, F Mentre, X Duval, D Descamps, J Timsit, Y Yazdanpanah, F Lescure, L Bouadma, D Nguyen, M Parisey, P Wicky, S Behillil, A Gaymard, M Bouscambert-Duchamp, F Donati, Q Le Hingrat, V Enouf, N Houhou-Fidouh, M Valette, A Mailles, J Lucet, F Mentre, X Duval, D Descamps, D Malvy, J Timsit, B Lina, S Van-Der-Werf, Y Yazdanpanah</b></div><div><b style="color: grey;">Results</b></div><div><p><span>opies per g of stool for patient 5. However, </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">the virus was not detected in the serum or the urine samples</span><span>. Patient 1 had nasopharyngeal samples collec</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/f8c15039fad3e71a1479b9d09516aad16fc52f70" target="_blank">Evaluation of SARS-CoV-2 RNA shedding in clinical specimens and clinical characteristics of 10 patients with COVID-19 in Macau</a></div><div><b>ILong Lo, CFu Lio, HHon Cheong, CIon Lei, &#61482; , H Cheong, X Zhong, Y Tian, N Sin</b></div><div><b style="color: grey;">Abstract</b></div><div><p><span>-2 RNA signals in all patients\' NPS and stool specimens but </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">negative in all urine specimens</span><span>. Based on our data on SARS-CoV-2 RNA shedding in stool and</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/712f1a74a5083d690832eed5fd9ff16a04434d8d" target="_blank">Rapid Detection of Novel Coronavirus (COVID-19) by Reverse Transcription- Loop-Mediated Isothermal Amplification</a></div><div><b>LE Lamb, SN Bartolone, E Ward, MB Chancellor</b></div><div><b style="color: grey;">Discussion</b></div><div><p><span /><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">the amount of virus present in an infected individual will likely vary between different biological specimens and over the time course of the infectio</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/3591382e6aefa066fe730ece11ba67995542353e" target="_blank">[No title available]</a></div><div><b>E Nicastri, A D&amp;apos;abramo, G Faggioni&#179;, R De Santis&#179;, A Mariano&#185;, L Lepore&#185;, F Molinari&#179;, G Petralito&#179;, S Fillo&#179;, D Munzi&#179;, A Corpolongo&#185;, L Bordi&#185;, F Carletti&#185;, C Castiletti&#185;, F Colavita&#185;, E Lalle&#185;, N Bevilacqua&#185;, MLetizia Giancola&#185;, L Scorzolini&#185;, S Lanini&#185;, C Palazzolo&#185;, A De Domenico&#8308;, MAnna Spinelli&#179;, P Scognamiglio&#185;, P Piredda&#8308;, R Iacomino&#8308;, A Mone&#8308;, V Puro&#185;, N Petrosillo&#185;, A Battistini&#8309;, F Vairo&#185;, F Lista, G Ippolito, N Emanuele, A D&amp;apos;, F Alessandra, D Giovanni, R Santis, M Andrea, L Luciana, M Filippo, P Giancarlo, F Silvia, M Diego, C Angela, B Licia, C Fabrizio, C Concetta, C Francesca, L Eleonora, B Nazario, M Giancola, S Letizia, L Laura, P Simone,  Claudia, D De,  Angelo, M Spinelli, S Anna, P Paola, I Paolo, M Raffaele, P Andrea, P Vincenzo, B Nicola, V Antonio, L Francesco, I Florigio,  Giuseppe</b></div><div><b style="color: grey;">Case description</b></div><div><p><span> resulted positive at different time points, whereas </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">urine, spermatic fluid, saliva, blood and conjunctival swabs were persistently negative </span><span>(Table) .</span></p></div><hr /></div><div><div><h1>Does the virus persist on fecal matter?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/f8c15039fad3e71a1479b9d09516aad16fc52f70" target="_blank">Evaluation of SARS-CoV-2 RNA shedding in clinical specimens and clinical characteristics of 10 patients with COVID-19 in Macau</a></div><div><b>ILong Lo, CFu Lio, HHon Cheong, CIon Lei, &#61482; , H Cheong, X Zhong, Y Tian, N Sin</b></div><div><b style="color: grey;">Discussion</b></div><div><p><span>te of SARS-CoV-2 deserves further evaluation since </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">the viability of SAR-CoV-2 in feces is uncertain</span><span>. Thus, we advocate that we should consider a combi</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/9b3c8ae64057fb199a1770ca46929ce031c34b56" target="_blank">Understanding the emerging coronavirus: what it means for health security and infection prevention</a></div><div><b>A Peters, P Vetter, C Guitart, N Lotfinejad, D Pittet</b></div><div><b style="color: grey;">What this means for infection prevention and control</b></div><div><p><span> that hospitals deal with on an annual basis, and </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">transmission seems to occur mainly through droplets</span><span>. Even a high estimate of the R 0 rate is many ti</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/dc0b621fdffe29eeeb88702249c64910c3bb51a3" target="_blank">Gastrointestinal tract symptoms in coronavirus disease 2019: Analysis of clinical symptoms in adult patients</a></div><div><b>Y Zhang, Z Lu, B Wang, J Cang, Y Ma</b></div><div><b style="color: grey;">DISCUSSIONS</b></div><div><p><span>e common in mild COVID-19. At present, </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">no large-scale study has discovered the SARS-CoV-2 virus in fecal samples</span><span>. 14 Therefore, a large sample size of</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/PMC7098032" target="_blank">Evaluation of SARS-CoV-2 RNA shedding in clinical specimens and clinical characteristics of 10 patients with COVID-19 in Macau</a></div><div><b>ILong Lo, CFu Lio, HHon Cheong, CIon Lei, THong Cheong, X Zhong, Y Tian, NNgan Sin</b></div><div><b style="color: grey;">Discussion</b></div><div><p><span>te of SARS-CoV-2 deserves further evaluation since </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">the viability of SAR-CoV-2 in feces is uncertain</span><span>. Thus, we advocate that we should consider a combi</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/PMC7104689" target="_blank">Cross-Country Comparison of Case Fatality Rates of COVID-19/SARS-COV-2</a></div><div><b>MAbdullatif Khafaie, F Rahim</b></div><div><b style="color: grey;">Introduction</b></div><div><p><span>eezing), as well as contact with infected surfaces. </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">COVID-19 virus can survive for days on surfaces</span><span>, but a simple disinfectant can eliminate this [2].</span></p></div><hr /></div><div><div><h1>Does the virus persist on blood?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/9bbfd3d34ee18ea1b9f4669331a6cee9c5992893" target="_blank">Virological assessment of hospitalized cases of coronavirus disease 2019 *equal contribution **senior authors with equal contribution</a></div><div><b>R W&#246;lfel, VM Corman, W Guggemos, M Seilmaier, S Zange, MA M&#252;ller, D Niemeyer, TC Jones Kelly, P Vollmar, C Rothe, M Hoelscher, T Bleicker, S Br&#252;nink, J Schneider, R Ehmann, K Zwirglmaier, C Drosten, C Wendtner, D; Katrin Zwirglmaier, K M&#252;nchen-Schwabing, G , D; Niemeyer,  Terence, J Kelly</b></div><div><b style="color: grey;">cases, providing proof of active virus replication in upper respiratory tract tissues.</b></div><div><p><span>m stool samples in spite of high virus RNA concentration. </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">Blood and urine never yielded virus</span><span>. Active replication in the throat was confirmed by viral</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/b741ebc747e054216b184b572bbe130bb9d772bd" target="_blank">Covid 19 pandemic and gynaecological laparoscopic surgery: knowns and unknowns</a></div><div><b>R Mallick, F Odejinmi, TJ Clark</b></div><div><b style="color: grey;">Recommendations for laparoscopic surgery in suspected or confirmed COVID-19</b></div><div><p><span> with ingestion into the gastrointestinal tract. However, </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">it is less commonly found in blood</span><span>, with studies reporting viral RNA positive samples in 1-1</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/3c66d0ad2c7daaf1f6a7a50f994b8c6830297fc3" target="_blank">[No title available]</a></div><div><b /></div><div><b style="color: grey;" /></div><div><p><span>es described (WNV, dengue, Zika), </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">the transmission by blood seems to be less efficient than the natural transmission</span><span>. Moreover, the majority of infect</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/53726a84035f04167440dde803026c996d8794c0" target="_blank">Global epidemiology, pathogenesis, immune response, diagnosis, treatment, economic and psychological impact, challenges, and future prevention of COVID-19: A Scoping review</a></div><div><b>B Berhe, H Legese, H Degefa, G Adhanom, A Gebrewahd, F Mardu, K Tesfay, M Welay, H Negash</b></div><div><b style="color: grey;">Pathogenesis and clinical syndrome of COVID 19</b></div><div><p><span>he respiratory tract 11 . After the virus reach in the lung </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">it spreads to peripheral blood</span><span>, causing viremia. Then the virus would adhere and express t</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/02d4c398a5509634beaf9cfd365697cf07d6b016" target="_blank">Journal Pre-proof Positive SARS-Cov-2 test in a woman with COVID-19 at 22 days after hospital discharge: A case report Positive SARS-Cov-2 test in a woman with COVID-19 at 22 days after hospital discharge: A case report</a></div><div><b>A Luo</b></div><div><b style="color: grey;">Discussion</b></div><div><p><span>ody tests on 17 March ( Table 1) . Instead of resurgence, </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">the virus was in fact still present</span><span>. According to the explanation of positive test results i</span></p></div><hr /></div><div><div><h1>Which materials does the virus persist on?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/9756bb3c608ed790d2306fc8db815a694eeca45f" target="_blank">Transmission routes of 2019-nCoV and controls in dental practice</a></div><div><b>X Peng, X Xu, Y Li, L Cheng, X Zhou, B Ren</b></div><div><b style="color: grey;">Airborne spread</b></div><div><p><span>ndemic human coronaviruses (HCoV) can persist on surfaces like </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">metal, glass, or plastic </span><span>for up to a couple of days 51, 56 . Therefore, contaminated su</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/78e49fdb6f0aa9924a5b510341d52b618fff0ca6" target="_blank">A Multiscale and Comparative Model for Receptor Binding of 2019 Novel Coronavirus and the Implication of its Life Cycle in Host Cells</a></div><div><b>Z Su, Y Wu</b></div><div><b style="color: grey;">Introduction</b></div><div><p><span>isease [1, 2] . It has been found that the replicated </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">RNA genome and synthesized viral proteins </span><span>are finally assembled together into 10 new viruses, be</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/be4df7a03de718c39fb660583a9be8accc895398" target="_blank">Rapid evidence summary on SARS-CoV-2 survivorship and disinfection, and a reusable PPE protocol using a double-hit process</a></div><div><b>JG B Derraik, WA Anderson, EA Connelly, YC Anderson, TPakari Child Health, W Trust, N Taranaki,  Zealand, Y Anderson</b></div><div><b style="color: grey;">Disinfection summary</b></div><div><p><span>ng both UVGI and heat treatment show a protective effect of </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">protein and aqueous substrata </span><span>on SARS-CoV-1 survival. &#8226; Unpublished experimental data from</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/0212e200dc708984bda32b474aa6a1437c6a3c3d" target="_blank">Risks of Novel Coronavirus Disease (COVID-19) in Preg- nancy; a Narrative Review</a></div><div><b>L Panahi, M Amiri, S Pouy</b></div><div><b style="color: grey;">Introduction</b></div><div><p><span>rmined, but the researchers found that the virus spreads through </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">respiratory droplets </span><span>like the flu, and air precautions are very necessary given the l</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/86c58960f3733a069909fe40cb2c20301e5f794b" target="_blank">Journal Pre-proof Anaesthesia and COVID-19: infection control Anaesthesia and COVID-19: infection control</a></div><div><b>PM Odor, M Neun, S Bampoe, S Clark, D Heaton, EM Hoogenboom, M Brown, A Patel, D Kamming</b></div><div><b style="color: grey;">Differences in the approach for COVID-19</b></div><div><p><span>Although the stability of aerosolised SARS-CoV-2 on </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">common environmental materials </span><span>appears to be similar to that of SARS-CoV in experimental condition</span></p></div><hr /></div><div><div><h1>How do people get infected with SARS-CoV-2?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/9f456a384cf88c9c204f80bfea4c0d50233875b2" target="_blank">A case report of neonatal COVID-19 infection in China</a></div><div><b>S Wang, L Guo, L Chen, W Liu, Y Cao, J Zhang, L Feng</b></div><div><b style="color: grey;">Discussion</b></div><div><p><span>ns have been reported in other provinces and cities in China [14] . The possibility of </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">mother-to-child transmission </span><span>of SARS-CoV-2 cannot be ruled out.</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/f3ff1ecae96700f41b83d2a034a3a959428388b0" target="_blank">participated in study design</a></div><div><b>JW A , JW C , GX , BP , YW , XY L , WF </b></div><div><b style="color: grey;">(which was not peer-reviewed)</b></div><div><p><span> old. Among three deaths, one was infected with SARS-CoV-2 </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">after recent lung cancer surgery</span><span>, one had diabetes, ischemic cardiomyopathy, and hyperthyro</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/c12d71462529856463c2432d5628d10a0027c404" target="_blank">Investigation of three clusters of COVID-19 in Singapore: implications for surveillance and response measures</a></div><div><b>R Pung, CJ Chiew, BE Young, S Chin, M Chen, HE Clapham, AR Cook, S Maurer-Stroh, MP H , S Toh, C Poh, M Low, J Lum, VT J Koh, TM Mak, L Cui, RV T P Lin, D Heng, Y Leo, DC Lye, VJ M Lee</b></div><div><b style="color: grey;">Cluster C</b></div><div><p><span>l history to China, who tested positive for SARS-CoV-2 from </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">enhanced pneumonia surveillance</span><span>. Activity mapping showed that both individuals had visited</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/9f456a384cf88c9c204f80bfea4c0d50233875b2" target="_blank">A case report of neonatal COVID-19 infection in China</a></div><div><b>S Wang, L Guo, L Chen, W Liu, Y Cao, J Zhang, L Feng</b></div><div><b style="color: grey;">Discussion</b></div><div><p><span>osis of intrauterine transmission, but the possibility of </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">vertical intrauterine transmission </span><span>of SARS-CoV-2 is not ruled out. The reasons are as follow</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/3348bf3dd5e93d966f677a72ef6de2322acd90dc" target="_blank">CORONA-steps for tracheotomy in COVID-19 patients: A staff-safe method for airway management</a></div><div><b>B Pichi, F Mazzola, A Bonsembiante, G Petruzzi, J Zocchi, S Moretto, A De Virgilio, R Pellini</b></div><div><b style="color: grey;">Introduction</b></div><div><p><span> since it began in late 2019 [1] . Due to the high virulence via </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">aerosol transmissions</span><span>, to date COVID-19 has infected more than 575,444 people all ove</span></p></div><hr /></div><div><div><h1>How do infected people transmit SARS-CoV-2?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/521479c589008ede74b15a4d4308ef79eefe39ca" target="_blank">Coronavirus disease 2019 (COVID-19): A literature review</a></div><div><b>H Harapan, N Itoh, A Yufika, W Winardi, S Keam, H Te, D Megawati, Z Hayati, AL Wagner, M Mudatsir, S Kuala, B Aceh, I </b></div><div><b style="color: grey;">Pathogenesis and immune response</b></div><div><p><span>ith ACE2 further support that SARS-CoV-2 has acquired capacity for </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">person-to-person </span><span>transmission [42] . Although, the spike protein sequence of recept</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/cf03feaa814c150f703e19c8aecae4ce5cf6f62c" target="_blank">Anesthetic Management of Patients Undergoing Aortic Dissection Repair With Suspected Severe Acute Respiratory Syndrome Coronavirus-2 Infection</a></div><div><b>H He, S Zhao, L Han, Q Wang, H Xia, X Huang, S Yao, J Huang, X Chen</b></div><div><b style="color: grey;" /></div><div><p><span> accumulating evidence has shown that SARS-CoV-2 exhibits effective </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">human-to-human </span><span>transmission, causing an exponential growth of new cases. 6, [10] [</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/9f456a384cf88c9c204f80bfea4c0d50233875b2" target="_blank">A case report of neonatal COVID-19 infection in China</a></div><div><b>S Wang, L Guo, L Chen, W Liu, Y Cao, J Zhang, L Feng</b></div><div><b style="color: grey;">Discussion</b></div><div><p><span>ns have been reported in other provinces and cities in China [14] . The possibility of </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">mother-to-child </span><span>transmission of SARS-CoV-2 cannot be ruled out.</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/2acb17f03d9fa51983bc975b159adbadc5f394a6" target="_blank">Journal Pre-proof EZH2-mediated H3K27me3 inhibits ACE2 expression EZH2-mediated H3K27me3 inhibits ACE2 expression</a></div><div><b>Y Li, H Li, L Zhou</b></div><div><b style="color: grey;" /></div><div><p><span>ociated with the SARS-CoV-2 and a retrospective study 44 indicated </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">person-to-person </span><span>transmission [5] . Moreover, people seem to be 45 generally suscep</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/PMC7098029" target="_blank">Perspectives on therapeutic neutralizing antibodies against the Novel Coronavirus SARS-CoV-2</a></div><div><b>G Zhou, Q Zhao</b></div><div><b style="color: grey;">Introduction</b></div><div><p><span>ence identity with SARS-CoV 2. Compared to SARS-CoV, transmitted </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">from human-to-human </span><span>of SARS-CoV-2 seems to be greater. As of February 2020, at least </span></p></div><hr /></div><div><div><h1>What diagnostics have been implemented for SARS-CoV-2?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/31816b72a378debeb42e76ff7ff2af69727cd049" target="_blank">Diagnosis of Acute Respiratory Syndrome Coronavirus 2 Infection by Detection of Nucleocapsid Protein running title: Diagnosis of COVID-19 by N antigen detection</a></div><div><b>B Diao, W Kun, J 2#,  Chen, Y Liu, Z Yuan, C Han, J Chen, Y Pan, &#65292; Chen, &#65292; Dan, J Wang, Y Chen, G Deng, H Zhou, Y Wu</b></div><div><b style="color: grey;">Discussion</b></div><div><p><span>event its spreading and clinical treatment. In practice, </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">nucleic acid test and antibody assay </span><span>have been employed in the diagnostic for suspected SARS-</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/aa5149a67e0b10602a6924b7f0d4ab77cc900c87" target="_blank">Comparisons of nucleic acid conversion time of SARS-CoV-2 of different samples in ICU and non-ICU patients Conversion time of SARS-CoV-2 RT-PCR in ICU and non-ICU patients Letter to the Editor Comparisons of nucleic acid conversion time of SARS-CoV-2 of different samples in ICU and non-ICU patients</a></div><div><b>Z Fang, Y Zhang, C Hang, W Zhang, J Ai, S Li</b></div><div><b style="color: grey;" /></div><div><p><span>sent. Total RNA was extracted from clinical samples and we performed </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">RT-PCR tests </span><span>targeting SARS-CoV-2. The continuous variants were described by mean</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/1e9cafd54968d935da52bdcae678df1e6ec2b650" target="_blank">SARS-CoV-2 specific antibody responses in COVID-19 patients</a></div><div><b>NM A Okba, MA M&#252;ller, W Li, C Wang, CH Geurtsvankessel, VM Corman, MM Lamers, RS Sikkema, E De Bruin, FD Chandler, Y Yazdanpanah, Q Le Hingrat, D Descamps, N Houhou-Fidouh, CB E M Reusken, B Bosch, C Drosten, MP G Koopmans, BL Haagmans</b></div><div><b style="color: grey;">Introduction</b></div><div><p><span>o the emergence of SARS-CoV-2 have led to the rapid availability of </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">real-time PCR </span><span>diagnostic assays that support the case ascertainment and tracking o</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/767515ad42d4761123f6fbe7a200c6dc3d8cf251" target="_blank">[No title available]</a></div><div><b>J Tesarik</b></div><div><b style="color: grey;">Background</b></div><div><p><span>ta have made it possible to produce specific diagnostic </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">polymerase chain reaction (PCR) tests </span><span>and will be useful in the future development of an effic</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/32b10bf8534bb8843320576bbf033d983e89f692" target="_blank">Letter to the editor: Plenty of coronaviruses but no SARS-CoV-2</a></div><div><b>P Colson, BLa Scola, V Esteves-Vieira&#185;, L Ninove, C Zandotti, M Jimeno&#8308;, C Gazin&#185;, M Bedotto&#185;, V Filosa&#185;, A Giraud-Gatineau, H Chaudet, P Brouqui, J Lagier, D Raoult, C Philippe, LScola Bernard, E Vera, N Laetitia, Z Christine, J Marie-Th&#233;r&#232;se, G C&#233;line, B Marielle, F V&#233;ronique, G Audrey, C Herv&#233;, B Philippe, J , R Didier</b></div><div><b style="color: grey;" /></div><div><p><span>ctious diseases, we have been implementing since the end of January 2020 </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">PCR </span><span>detection of SARS-CoV-2 RNA using several systems, including those releas</span></p></div><hr /></div><div><div><h1>What models for infection exist for SARS-CoV-2?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/49f4c2847c6701324210ee8ab2c8878928554527" target="_blank">Microneedle array delivered recombinant coronavirus vaccines: Immunogenicity and rapid translational development</a></div><div><b>E Kim, G Erdos, S Huang, TW Kenniston, SC Balmert, C Donahue Carey, V Stalin Raj, MW Epperly, WB Klimstra, BL Haagmans, E Korkmaz, LD Falo, A Gambotto, LD Falo, A Gambotto, ) </b></div><div><b style="color: grey;">Introduction</b></div><div><p><span>ns for vaccine design and for predicting pandemic potential. </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">Human-to-human transmission </span><span>of SARS-CoV-2 has now been established, and the situation wit</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/a1ddaf941a4c1a0145ce1af098cf1d9da207ed35" target="_blank">Broad Spectrum Antiviral Agent Niclosamide and Its Therapeutic Potential</a></div><div><b>J Xu, P Shi, H Li, J Zhou</b></div><div><b style="color: grey;" /></div><div><p><span>ty and mortality. 3 While coronaviruses are often zoonotic, </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">person-to-person transmission </span><span>has been confirmed for SARS-CoV-2, similar to MERS-CoV and S</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/85783a36e7e787302307f42460839435d665f4e7" target="_blank">SARS-CoV-2: an Emerging Coronavirus that Causes a Global Threat</a></div><div><b>J Zheng</b></div><div><b style="color: grey;">Chemotherapeutic options for SARS-CoV-2 infection</b></div><div><p><span> and favipiravir (T-750) against the infection of SARS-CoV-2 on </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">Vero E6 cells in vitro </span><span>[63] . Among these seven drugs, chloroquine and remdesivir demo</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/eb7fc5004191e8fce0d6e4ff98de6865872e665f" target="_blank">Spike protein binding prediction with neutralizing antibodies of SARS-CoV-2 1 2</a></div><div><b> Republic,  Korea</b></div><div><b style="color: grey;">214</b></div><div><p><span>nerated not only the crystal structures of SARS-216 CoV and MERS-CoV S-RBD proteins, but also the </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">high-quaility homology models </span><span>with SARS-CoV-2 S-RBD.</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/c3c1f7896932b6295deccd52b23336f3149720fc" target="_blank">Stochastic Compartmental Modelling of SARS-CoV-2 with Approximate Bayesian Computation</a></div><div><b>V Chandra</b></div><div><b style="color: grey;">DISCUSSION</b></div><div><p><span>se higher-dimensional parameter spaces. Specific to SARS-CoV-2, </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">age-structured models </span><span>would also be a valuable development, as would models that inclu</span></p></div><hr /></div><div><div><h1>What models for transmission exist for SARS-CoV-2?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/521479c589008ede74b15a4d4308ef79eefe39ca" target="_blank">Coronavirus disease 2019 (COVID-19): A literature review</a></div><div><b>H Harapan, N Itoh, A Yufika, W Winardi, S Keam, H Te, D Megawati, Z Hayati, AL Wagner, M Mudatsir, S Kuala, B Aceh, I </b></div><div><b style="color: grey;">Pathogenesis and immune response</b></div><div><p><span>ith ACE2 further support that SARS-CoV-2 has acquired capacity for </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">person-to-person </span><span>transmission [42] . Although, the spike protein sequence of recept</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/a1ddaf941a4c1a0145ce1af098cf1d9da207ed35" target="_blank">Broad Spectrum Antiviral Agent Niclosamide and Its Therapeutic Potential</a></div><div><b>J Xu, P Shi, H Li, J Zhou</b></div><div><b style="color: grey;" /></div><div><p><span>morbidity and mortality. 3 While coronaviruses are often zoonotic, </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">person-to-person </span><span>transmission has been confirmed for SARS-CoV-2, similar to MERS-Co</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/cf03feaa814c150f703e19c8aecae4ce5cf6f62c" target="_blank">Anesthetic Management of Patients Undergoing Aortic Dissection Repair With Suspected Severe Acute Respiratory Syndrome Coronavirus-2 Infection</a></div><div><b>H He, S Zhao, L Han, Q Wang, H Xia, X Huang, S Yao, J Huang, X Chen</b></div><div><b style="color: grey;" /></div><div><p><span> accumulating evidence has shown that SARS-CoV-2 exhibits effective </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">human-to-human </span><span>transmission, causing an exponential growth of new cases. 6, [10] [</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/49f4c2847c6701324210ee8ab2c8878928554527" target="_blank">Microneedle array delivered recombinant coronavirus vaccines: Immunogenicity and rapid translational development</a></div><div><b>E Kim, G Erdos, S Huang, TW Kenniston, SC Balmert, C Donahue Carey, V Stalin Raj, MW Epperly, WB Klimstra, BL Haagmans, E Korkmaz, LD Falo, A Gambotto, LD Falo, A Gambotto, ) </b></div><div><b style="color: grey;">Introduction</b></div><div><p><span>ns for vaccine design and for predicting pandemic potential. </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">Human-to-human transmission </span><span>of SARS-CoV-2 has now been established, and the situation wit</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/49f4c2847c6701324210ee8ab2c8878928554527" target="_blank">Microneedle array delivered recombinant coronavirus vaccines: Immunogenicity and rapid translational development</a></div><div><b>E Kim, G Erdos, S Huang, TW Kenniston, SC Balmert, C Donahue Carey, V Stalin Raj, MW Epperly, WB Klimstra, BL Haagmans, E Korkmaz, LD Falo, A Gambotto, LD Falo, A Gambotto, ) </b></div><div><b style="color: grey;">Introduction</b></div><div><p><span>lications for vaccine design and for predicting pandemic potential. </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">Human-to-human </span><span>transmission of SARS-CoV-2 has now been established, and the situat</span></p></div><hr /></div><div><div><h1>What studies exist about monitoring phenotypic change in SARS-CoV-2?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/2892aee133b8b4037ace7ec6dfe6cab17e466292" target="_blank">[No title available]</a></div><div><b>E Petersen, D Hui, DH Hamer, L Blumberg, LC Madoff, M Pollack, S Shui,  Lee, S Mclellan, Z Memish, I Praharaj, S Wasserman, F Ntoumi, EIbraheem Azhar, TD Mchugh, R Kock, G Ippolito, M Koopmans</b></div><div><b style="color: grey;" /></div><div><p><span> which was rapidly identified after these clinicians sent samples to a reference laboratory for </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">next generation sequencing </span><span>(NGS) (Zhou et al., 2020) .</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/fc520c33ec7052adb24af0b3bbb43526f92f6eac" target="_blank">[No title available]</a></div><div><b>E Petersen, D Hui, DH Hamer, L Blumberg, LC Madoff, M Pollack, S Shui,  Lee, S Mclellan, Z Memish, I Praharaj, S Wasserman, F Ntoumi, EIbraheem Azhar, TD Mchugh, R Kock, G Ippolito, M Koopmans</b></div><div><b style="color: grey;">Editorial</b></div><div><p><span> which was rapidly identified after these clinicians sent samples to a reference laboratory for </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">next generation sequencing </span><span>(NGS) (Zhou et al., 2020) .</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/6dbe863788c7a02efb68c5a88287484cf00c29f9" target="_blank">[No title available]</a></div><div><b /></div><div><b style="color: grey;">ARTICLE IN PRESS</b></div><div><p><span>ggest that SARS-CoV-2 research should focus on the establishment of </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">animal models </span><span>that recapitulate the various aspects of human disease and determina</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/df97f804b68dcf16ffcc3c2c0894528d6d0c7ff2" target="_blank">Articles The effect of control strategies to reduce social mixing on outcomes of the COVID-19 epidemic in Wuhan, China: a modelling study</a></div><div><b>K Prem, Y Liu, TW Russell, AJ Kucharski, RM Eggo, N Davies</b></div><div><b style="color: grey;">Research in context</b></div><div><p><span>ledge, </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">no published article has reported use of location-specific transmission models that consider the impacts of school or workplace closures </span><span>to stu</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/8c210df7072bc88fa7d2da72b25c84430c16f5e8" target="_blank">Recombination and lineage-specific mutations led to the emergence of SARS-CoV-2</a></div><div><b>J&#193;ngel Pati&#241;o-Galindo, I 2+,  Filip, M 2+,  Alquraishi, R Rabadan</b></div><div><b style="color: grey;">Introduction</b></div><div><p><span>ARS-CoV-2 to share a similar haplotype in the RBD. Through </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">ancestral reconstruction analyses</span><span>, we also observe that, subsequent to this recombination, </span></p></div><hr /></div><div><div><h1>What studies exist about monitoring adaptation of SARS-CoV-2?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/f863c592acdbb85b20cfd781082d3ba883dbc8f7" target="_blank">[No title available]</a></div><div><b /></div><div><b style="color: grey;">Enteric involvement of coronaviruses: is faecal-oral transmission of SARS-CoV-2 possible?</b></div><div><p><span>dity, although viability decreased to 8 h at 30&#176;C and 80% relative humidity conditions. 14 At present, </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">no viability data </span><span>are available for SARS-CoV-2.</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/3ef0df1127178cb56f40a4e4fcb6a39b82df6730" target="_blank">Widespread use of face masks in public may slow the spread of SARS CoV-2: 1 an ecological study 2 3 4</a></div><div><b>C Kenyon</b></div><div><b style="color: grey;">88</b></div><div><p><span>on played an important role in the outbreak [10, 11] . Likewise in </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">93 vitro studies </span><span>demonstrate that SARS CoV-2 can be aerosolized and remain viable i</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/6dbe863788c7a02efb68c5a88287484cf00c29f9" target="_blank">[No title available]</a></div><div><b /></div><div><b style="color: grey;">ARTICLE IN PRESS</b></div><div><p><span>ggest that SARS-CoV-2 research should focus on the establishment of </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">animal models </span><span>that recapitulate the various aspects of human disease and determina</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/49f4c2847c6701324210ee8ab2c8878928554527" target="_blank">Microneedle array delivered recombinant coronavirus vaccines: Immunogenicity and rapid translational development</a></div><div><b>E Kim, G Erdos, S Huang, TW Kenniston, SC Balmert, C Donahue Carey, V Stalin Raj, MW Epperly, WB Klimstra, BL Haagmans, E Korkmaz, LD Falo, A Gambotto, LD Falo, A Gambotto, ) </b></div><div><b style="color: grey;">Discussion</b></div><div><p><span>st by considerably reducing the required vaccine doses [58] . The </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">stability studies </span><span>with our MNA SARS-CoV-2 vaccines are currently in progress, howeve</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/00c71f72eb837e2342dbda8761664e221e2a03df" target="_blank">Intensive care management of coronavirus disease 2019 (COVID-19): challenges and recommendations</a></div><div><b>L Weng, C Lim, Y Koh, J Phua, L Weng, L Ling, M Egi, C Lim, V Divatia, BRaja Shrestha, YM Arabi, J Ng, CD Gomersall, M Nishimura, Y Koh</b></div><div><b style="color: grey;">Intravenous immunoglobulin (immunotherapy)</b></div><div><p><span>e of controls 85 </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">No peer-reviewed, published safety data available for SARS-CoV-2, but studies of SARS-CoV have not reported serious adverse events </span><span>83</span></p></div><hr /></div><div><div><h1>How is the immune response to SARS-CoV-2?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/51284f48a669e302ca8d6126d0aa410e54ba9c44" target="_blank">COVID-19: Melatonin as a potential adjuvant treatment</a></div><div><b>R Zhang, X Wang, L Ni, X Di, B Ma, S Niu, C Liu, RJ Reiter</b></div><div><b style="color: grey;">Pathogenesis of COVID-19 and the rationale for melatonin use</b></div><div><p><span>r apoptosis or We postulated that lungs infected by SARS-CoV-2, and a </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">suppressed </span><span>immune response, elevated inflammation and excessive oxidation stress</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/f616a4bfbfb135e9709d553e13fff682cbbd71f3" target="_blank">Targeting the catecholamine-cytokine axis to prevent SARS-CoV-2 cytokine storm syndrome</a></div><div><b>MF Konig, M Powell, V Staedtke, R Bai, DL Thomas, N Fischer, S Huq, AM Khalafallah, A Koenecke, N Papadopoulos, KW Kinzler, B Vogelstein, JT Vogelstein, S Athey, S Zhou, C Bettegowda</b></div><div><b style="color: grey;" /></div><div><p><span>to be driven by acute respiratory distress syndrome (ARDS) and a </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">dysregulated </span><span>immune response to SARS-CoV-2 2-4 .</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/ee815a83e26fb560ad5648af7f2fc592c9c9732e" target="_blank">Persistent SARS-CoV-2 presence is companied with defects in adaptive immune system in non-severe COVID-19 patients</a></div><div><b>B Liu, J Han, X Cheng, L Zhang, ; , W Wang, L Ni, C Wei, Y Huang, Z Cheng</b></div><div><b style="color: grey;">Abstract</b></div><div><p><span>nce in non-severe COVID-19 patients is associated with </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">reduced numbers of adaptive immune cells</span><span>. Monitoring lymphocyte subpopulations could be clinica</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/PMC7113449" target="_blank">COVID-19, A Clinical Syndrome Manifesting as Hypersensitivity Pneumonitis</a></div><div><b>YGoo Song, H Shin</b></div><div><b style="color: grey;" /></div><div><p><span>Acute HP is more common in the elderly. The </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">defective </span><span>innate immunity and the accelerated complement activation induced by SARS-CoV-2, similar to that</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/9dd215f5b70cf477742426fffa12fbf95a45a66c" target="_blank">The global battle against SARS-CoV-2 and COVID-19</a></div><div><b>C Deng</b></div><div><b style="color: grey;" /></div><div><p><span> Zhou and Zhao pointed out the great importance of using </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">therapeutic neutralizing antibodies </span><span>(NAbs) to control the spread and re-emergence of SARS-CoV</span></p></div><hr /></div><div><div><h1>How is immunity to SARS-CoV-2 developed?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/4e4797b808c48e3b9514b3f598ebcff6a598eddc" target="_blank">Comparative Genomic Analysis of Rapidly Evolving SARS CoV-2 Viruses Reveal Mosaic Pattern of Phylogeographical Distribution</a></div><div><b>R Kumar, H Verma, N Singhvi, U Sood, V Gupta, M Singh, R Kumari, P Hira, S Nagar, C Talwar, N Nayyar, S Anand, C Dogra Rawat, M Verma, RKrishan Negi, Y Singh, R Lal</b></div><div><b style="color: grey;">Abstract</b></div><div><p><span>,500 lives and has impacted human lives worldwide. Development of </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">universal vaccines </span><span>against the novel SARS-CoV-2 holds utmost urgency to control COVI</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/734779fad93249a2f6fede6afd10eeff7b37919b" target="_blank">Projecting the transmission dynamics of SARS-CoV-2 through the post-pandemic period</a></div><div><b>SM Kissler, &#8224; , C Tedijanto, EM Goldstein, YH Grad, M Lipsitch</b></div><div><b style="color: grey;">Abstract</b></div><div><p><span> data still needed to distinguish between them, most importantly </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">longitudinal serological studies </span><span>to determine the duration of immunity to SARS-CoV-2.</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/cd4f5118038ae33283d34a15f8c3888f00e247bc" target="_blank">The deadly coronaviruses: The 2003 SARS pandemic and the 2020 novel coronavirus epidemic in China</a></div><div><b>Y Yang, F Peng, R Wang, K Guan, T Jiang, G Xu, J Sun, C Chang</b></div><div><b style="color: grey;">Development of vaccines for coronavirus</b></div><div><p><span>ccine during the outbreak. Development of </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">animal models that faithfully mimic SARS-CoV-2 infection in humans </span><span>is critical not only in providing a more </span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/ac8c98a982e2b5aec7061105538e56fcaedeaf85" target="_blank">Epidemiological Tools that Predict Partial Herd Immunity to SARS Coronavirus 2</a></div><div><b>Y Kamikubo, A Takahashi, YK </b></div><div><b style="color: grey;">Although traditional public health outbreak response tactics implemented in</b></div><div><p><span /><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">Quantitative analysis </span><span>for SARS-CoV-2-specific IgM and IgG antibodies in the blood of healthy persons may reveal herd immunity against S type SARS-CoV-</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/655b175741b74ae8c98f93fd449c88a147bee96e" target="_blank">): e35 REVIEW ARTICLE A. Prevalence of Underlying Diseases in Hospitalized Patients with COVID-19: a Systematic Review and Meta-Analysis</a></div><div><b>A Emami, F Javanmardi, N Pirbonyeh, A Akbari</b></div><div><b style="color: grey;">Discussion</b></div><div><p><span>hat put patients at higher risk of SARS-CoV-2 threats. </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">Decreasing the proinflammatory cytokines</span><span>, which leads to a weaker immune function may account f</span></p></div><hr /></div><div><div><h1>Are movement control strategies effective to prevent secondary transmission of SARS-CoV-2?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/0bd443591b5e10934beef050b516afeca6b668fe" target="_blank">A simple model to assess Wuhan lock-down effect and region efforts during COVID-19 epidemic in China Mainland</a></div><div><b>Z Yuan, Y Xiao, Z Dai, J Huang, Y Chen</b></div><div><b style="color: grey;">Since Corona Virus Disease 2019 (COVID-19) caused by SARS-CoV-2, a novel coronavirus, emerged on</b></div><div><p><span>ever, Read et al (2020) suggested that </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">travel restrictions from and to Wuhan city are unlikely to be effective </span><span>in halting transmission across China; w</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/58b5c77c9fb3f68a3ad84a3f15275dc0e4554192" target="_blank">To appear in: One Health</a></div><div><b>ME El Zowalaty, JD J&#228;rhult</b></div><div><b style="color: grey;">J o u r n a l P r e -p r o o f</b></div><div><p><span>nce of airborne transmission of the SARS-CoV-2, however </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">precautionary measures are recommended </span><span>due to the lack of information excluding this mode of t</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/b2609e4c3da56065b800d735bac4a0dde1653489" target="_blank">From SARS to COVID-19: A previously unknown SARS-related coronavirus (SARS-CoV-2) of pandemic potential infecting humans -Call for a One Health approach</a></div><div><b>ME El Zowalaty, JD J&#228;rhult</b></div><div><b style="color: grey;" /></div><div><p><span>nce of airborne transmission of the SARS-CoV-2, however </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">precautionary measures are recommended </span><span>due to the lack of information excluding this mode of t</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/e3fa5ac28a1cd0b48cccb80d7ab65cdc429b08ac" target="_blank">From SARS to COVID-19: A previously unknown SARS-related coronavirus (SARS-CoV-2) of pandemic potential infecting humans -Call for a One Health approach</a></div><div><b>ME El Zowalaty, JD J&#228;rhult</b></div><div><b style="color: grey;" /></div><div><p><span>nce of airborne transmission of the SARS-CoV-2, however </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">precautionary measures are recommended </span><span>due to the lack of information excluding this mode of t</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/7c4cf4696e04c54841d693ab185b9fd22e235e1d" target="_blank">China after wartime control measures: a population-level retrospective study</a></div><div><b>K Wang, J Gao, H Wang, X Wu, Q Yuan, F Guo, Z Zhang, Y Cheng, K Wang, GJ , WH , W , Y , G , Z Cheng, Y </b></div><div><b style="color: grey;">Discussion</b></div><div><p><span>ifficult. Therefore, </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">wartime control measures for COVID-2019, including limited population movements, were coercive but feasible </span><span>[14] . One reported c</span></p></div><hr /></div><div><div><h1>Is personal protective equipment effective at reducing risk of transmission of SARS-CoV-2?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/a3d3c0ad321f91f618949f0ed02762d85a3bac44" target="_blank">[No title available]</a></div><div><b>DN Maxwell, TM Perl,  Fshea, JB Cutrell, H Hines, B Dallas</b></div><div><b style="color: grey;" /></div><div><p><span>gative pressure room, </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">no advanced personal protective equipment (PPE) such as powered air-purifying respirators (PAPRs) was used</span><span>. The rationale was si</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/bb659ff1b0d45d4e17e10268f4d1daaca3491269" target="_blank">Journal Pre-proof Running a Radiation Oncology Department at the time of coronavirus: an Italian experience Title: Running a Radiation Oncology Department at the time of coronavirus: an Italian experience Short title: Radiation Oncology at the time of coronavirus Authors: Title: Running a Radiation Oncology Department at the time of coronavirus: an Italian experience</a></div><div><b>M Krengli, E Ferrara, F Mastroleo, M Brambilla, U Ricardi</b></div><div><b style="color: grey;">Timeline of spread of the virus and Health Ministry recommendations</b></div><div><p><span>. The patient has been treated by health professionals with </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">no adequate protective devices</span><span>. This was the beginning of the spread of affected patients,</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/PMC7137533" target="_blank">Coronavirus disease 2019 (COVID-19) outbreak in Iran: Actions and problems</a></div><div><b>M Abdi</b></div><div><b style="color: grey;" /></div><div><p><span>d per capita hospital beds and equipment in some cities.(2)</span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">Inadequate protective equipment </span><span>such as facial masks, disinfectants, and antiseptics such a</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/754315299d847600d6c5d414665c728d40bf731d" target="_blank">Breaking down of healthcare system: Mathematical modelling for controlling the novel coronavirus (2019-nCoV) outbreak in Wuhan, China</a></div><div><b>W -Kit, M 1#, J Huang, CJ P Zhang 3#,  Wai-Kit,  Ming</b></div><div><b style="color: grey;">Discussion</b></div><div><p><span>ng events. </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">Personal prevention strategies for seasonal influenza and other viral infections are still applicable during the present outbreak</span><span>, inclusiv</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/PMC7136995" target="_blank">First statement on preparation for the COVID-19 pandemic in large German Speaking University-based radiation oncology departments</a></div><div><b>SE. Combs, C Belka, M Niyazi, S Corradini, S Pigorsch, J Wilkens, AL. Grosu, M Guckenberger, U Ganswindt, D Bernhardt</b></div><div><b style="color: grey;">Introduction</b></div><div><p><span>t [28, 29]. In cases of SARS-CoV-2 positivity, </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">a more advanced personal protection equipment is needed </span><span>and includes: disposable overalls (tunics and/o</span></p></div><hr /></div><div><div><h1>Is transmission of SARS-CoV-2 dependent on environmental factors?</h1></div><div><a href="https://cord-19.apps.allenai.org/paper/4cd306cff8ce22bd079af8ad5750aae567d78126" target="_blank">Journal Pre-proof How to perform tracheal intubation for COVID-19 patients safely and effectively: Experience AND tips from Sichuan, China How to Perform Tracheal Intubation for COVID-19 Patients Safely and Effectively: Experience AND Tips from Sichuan, China</a></div><div><b>J Huang, J Zeng, M Xie, X Huang, X Wei, L Pan, &#65292; Huang</b></div><div><b style="color: grey;" /></div><div><p><span>firmed cases around multiple countries every day. </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">The transmission dynamics is not fully understood </span><span>[1] . Establishing an artificial airway is a techn</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/e1200121af466e437f3cf17aa10654b7725edb07" target="_blank">[No title available]</a></div><div><b>C Massonnaud, J Roux, P Cr&#233;pey</b></div><div><b style="color: grey;">Discussion</b></div><div><p><span> the following winter. However, </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">whether SARS-CoV-2 transmission will be affected by seasonal variations remains unclear</span><span>. Although many infectious dise</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/1989bf8f3de6079242e304cf7fe487f2900a9bbe" target="_blank">[No title available]</a></div><div><b /></div><div><b style="color: grey;" /></div><div><p><span /><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">Whether SARS-CoV-2 is viable under environmental conditions that could facilitate faecal-oral transmission is not yet clear</span><span>. However, evidence exists </span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/86c58960f3733a069909fe40cb2c20301e5f794b" target="_blank">Journal Pre-proof Anaesthesia and COVID-19: infection control Anaesthesia and COVID-19: infection control</a></div><div><b>PM Odor, M Neun, S Bampoe, S Clark, D Heaton, EM Hoogenboom, M Brown, A Patel, D Kamming</b></div><div><b style="color: grey;">Differences in the approach for COVID-19</b></div><div><p><span>ividuals. Data on the human infectious dose by aerosol and surface contact (fomite) </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">transmission routes are still lacking </span><span>for the SARS-CoV-2 pathogen.</span></p></div><hr /><div><a href="https://cord-19.apps.allenai.org/paper/f294f0df7468a8ac9e27776cc15fa20297a9f040" target="_blank">Systematic Comparison of Two Animal-to-Human Transmitted Human Coronaviruses: SARS-CoV-2 and SARS-CoV</a></div><div><b>J Xu, S Zhao, T Teng, AElgaili Abdalla, W Zhu, L Xie, Y Wang, X Guo</b></div><div><b style="color: grey;">Prevention</b></div><div><p><span>ia droplets or close contacts [58, 59] , but </span><span style="background-color: #DCDCDC; border-radius: 5px; padding: 5px;">aerosol and fecal-oral transmission still need further study </span><span>[60, 61] . To reduce virus transmission, ear</span></p></div><hr /></div>